<a href="https://colab.research.google.com/github/jerryshenfewcher/Cits4012/blob/main/ABSA_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Abstract

Import data from Google Drive

In [57]:
# Download from Github

!wget https://raw.githubusercontent.com/jerryshenfewcher/Cits4012/main/test.json
!wget https://raw.githubusercontent.com/jerryshenfewcher/Cits4012/main/train.json
!wget https://raw.githubusercontent.com/jerryshenfewcher/Cits4012/main/val.json

!ls

--2024-04-30 02:27:16--  https://raw.githubusercontent.com/jerryshenfewcher/Cits4012/main/test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141274 (138K) [text/plain]
Saving to: ‘test.json.2’

test.json.2         100%[===================>] 137.96K  --.-KB/s    in 0.03s   

2024-04-30 02:27:17 (3.98 MB/s) - ‘test.json.2’ saved [141274/141274]

--2024-04-30 02:27:17--  https://raw.githubusercontent.com/jerryshenfewcher/Cits4012/main/train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1136800 (1.1M) [text/plain]
Saving

In [58]:
import json
import pandas as pd

# 遍历文件列表，加载每个文件
def fileopen(file):
  with open(file, 'r') as f:
    data = json.load(f)
    return data

test_data = fileopen('test.json')
train_data = fileopen('train.json')
val_data = fileopen('val.json')

print(test_data)
print(train_data)
print(val_data)

{'columns': ['sentence', 'aspect', 'polarity'], 'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 21

Data Preprocess

In [59]:
import re
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# These are just common English contractions. We used it in Lab 5 before!
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def pre_process(data):
    output = []
    for sent_list in data["data"]:
        sent = sent_list[0].lower()  # Assuming the first element is the sentence
        for word, new_word in contraction_dict.items():
            sent = sent.replace(word, new_word)  # Dealing with contractions
        sent = re.sub(r'[^\w\s]', '', sent)  # Removing punctuation
        tokens = word_tokenize(sent)  # Tokenization
        aspect = sent_list[1]
        polarity = sent_list[2]
        output.append([tokens, [aspect], [polarity]])
    return output

# 使用示例
test_token_list = pre_process(test_data)
train_token_list = pre_process(train_data)
val_token_list = pre_process(val_data)

print(test_token_list)
print(train_token_list)
print(val_token_list)

# We use zip (* pre process (data)) to split the output of the pre process function into three separate lists, each containing review, aspect, and polarity information.
test_token_list, test_aspect_list, test_polarity_list = zip(*test_token_list)
train_token_list, train_aspect_list, train_polarity_list = zip(*train_token_list)
val_token_list, val_aspect_list, val_polarity_list = zip(*val_token_list)

# Transfer from tuple to list
test_token_list = list(test_token_list)
test_aspect_list = list(test_aspect_list)
test_polarity_list = list(test_polarity_list)

train_token_list = list(train_token_list)
train_aspect_list = list(train_aspect_list)
train_polarity_list = list(train_polarity_list)

val_token_list = list(val_token_list)
val_aspect_list = list(val_aspect_list)
val_polarity_list = list(val_polarity_list)

print(test_token_list)
print(test_aspect_list)
print(test_polarity_list)

# Adding special tokens for generation: <BOS> indicates beginning of sentence, <EOS> means end o sentence.
output_test_polarity_list = [["<BOS>"] + s for s in test_polarity_list] # -> input to decoder during the teacher-forcing training
target_test_polarity_list = [s + ["<EOS>"] for s in test_polarity_list] # -> ground truth to be compared with the decoder output

output_train_polarity_list = [["<BOS>"] + s for s in train_polarity_list]
target_train_polarity_list = [s + ["<EOS>"] for s in train_polarity_list]

output_val_polarity_list = [["<BOS>"] + s for s in val_polarity_list]
target_val_polarity_list = [s + ["<EOS>"] for s in val_polarity_list]

print(output_test_polarity_list)
print(target_test_polarity_list)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[[['we', 'went', 'again', 'and', 'sat', 'at', 'the', 'bar', 'this', 'time', 'i', 'had', '5', 'pints', 'of', 'guinness', 'and', 'not', 'one', 'buyback', 'i', 'ordered', 'a', 'basket', 'of', 'onion', 'rings', 'and', 'there', 'were', 'about', '5', 'in', 'the', 'basket', 'the', 'rest', 'was', 'filled', 'with', 'crumbs', 'the', 'chili', 'was', 'not', 'even', 'edible'], ['place'], ['neutral']], [['we', 'went', 'again', 'and', 'sat', 'at', 'the', 'bar', 'this', 'time', 'i', 'had', '5', 'pints', 'of', 'guinness', 'and', 'not', 'one', 'buyback', 'i', 'ordered', 'a', 'basket', 'of', 'onion', 'rings', 'and', 'there', 'were', 'about', '5', 'in', 'the', 'basket', 'the', 'rest', 'was', 'filled', 'with', 'crumbs', 'the', 'chili', 'was', 'not', 'even', 'edible'], ['food'], ['negative']], [['the', 'food', 'was', 'good', 'but', 'it', 'is', 'not', 'worth', 'the', 'waitor', 'the', 'lousy', 'service'], ['food'], ['positive']], [['the', 'food', 'was', 'good', 'but', 'it', 'is', 'not', 'worth', 'the', 'waito

In [60]:
# Here we simply use the maximum sentence length
MAX_LENGTH = max([len(s) for s in test_token_list] + [len(s) for s in target_test_polarity_list])

print(MAX_LENGTH)

64


In [61]:
# set up a vocab to index dictionary
word_to_ix = {"<BOS>": 0, "<EOS>":1}

for sentence in test_token_list+output_test_polarity_list:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

print(word_list)

['<BOS>', '<EOS>', 'we', 'went', 'again', 'and', 'sat', 'at', 'the', 'bar', 'this', 'time', 'i', 'had', '5', 'pints', 'of', 'guinness', 'not', 'one', 'buyback', 'ordered', 'a', 'basket', 'onion', 'rings', 'there', 'were', 'about', 'in', 'rest', 'was', 'filled', 'with', 'crumbs', 'chili', 'even', 'edible', 'food', 'good', 'but', 'it', 'is', 'worth', 'waitor', 'lousy', 'service', 'waiter', 'took', 'our', 'drink', 'order', 'then', 'did', 'see', 'him', 'for', '15', 'minutes', 'does', 'get', 'crowded', 'plenty', 'blue', 'shirted', 'cigar', 'smoking', 'wannabes', 'so', 'make', 'reservation', 'after', 'hearing', 'all', 'specials', 'you', 'would', 'think', 'they', 'twice', 'as', 'much', 'menu', 'prices', 'other', 'places', 'do', 'waitress', 'never', 'came', 'back', 'to', 'table', 'when', 'down', 'barely', 'looked', 'direction', 'abruptly', 'shoved', 'menus', 'on', 'continue', 'expand', 'their', 'garden', 'space', 'wout', 'expanding', 'kitchen', 'can', 'provide', 'adequate', 'extremely', 'rude'

In [62]:
# construct token index lists for input, output and target
# i.e., to convert each sequence into corresponding index based on the index dictionary we created in the previous section
# e.g., ['are', 'your', 'systems', 'functioning'] -> [42, 6, 2576, 2577]
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

input_index = to_index(test_token_list, word_to_ix)
output_index = to_index(output_test_polarity_list, word_to_ix)
target_index = to_index(target_test_polarity_list, word_to_ix)

print(input_index)
print(output_index)
print(target_index)

[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 5, 18, 19, 20, 12, 21, 22, 23, 16, 24, 25, 5, 26, 27, 28, 14, 29, 8, 23, 8, 30, 31, 32, 33, 34, 8, 35, 31, 18, 36, 37], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 5, 18, 19, 20, 12, 21, 22, 23, 16, 24, 25, 5, 26, 27, 28, 14, 29, 8, 23, 8, 30, 31, 32, 33, 34, 8, 35, 31, 18, 36, 37], [8, 38, 31, 39, 40, 41, 42, 18, 43, 8, 44, 8, 45, 46], [8, 38, 31, 39, 40, 41, 42, 18, 43, 8, 44, 8, 45, 46], [47, 48, 49, 50, 51, 5, 52, 2, 53, 18, 54, 55, 56, 57, 58], [47, 48, 49, 50, 51, 5, 52, 2, 53, 18, 54, 55, 56, 57, 58], [41, 59, 60, 61, 33, 62, 16, 63, 64, 65, 66, 67, 68, 69, 22, 70], [41, 59, 60, 61, 33, 62, 16, 63, 64, 65, 66, 67, 68, 69, 22, 70], [71, 72, 73, 16, 8, 74, 75, 76, 77, 78, 27, 79, 80, 81, 80, 8, 82, 83, 80, 84, 85, 86], [71, 72, 73, 16, 8, 74, 75, 76, 77, 78, 27, 79, 80, 81, 80, 8, 82, 83, 80, 84, 85, 86], [49, 87, 48, 49, 51, 5, 52, 88, 89, 90, 91, 49, 92], [49, 87, 48, 49, 51, 5, 52, 88, 89, 90, 91, 49, 92], [

 # + 异常处理，+ 数据验证步骤

# Model

In [63]:
import torch
import torch
import torch.nn as nn
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [64]:
# 示例词汇表
word_to_ix = {"<BOS>": 0, "<EOS>": 1, "hello": 2, "world": 3}
hidden_size = 50

# 创建 embedding 层
embedding = nn.Embedding(len(word_to_ix), hidden_size).to(device)

# 初始化 encoder 和 decoder
encoder1 = EncoderRNN(hidden_size, embedding).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, len(word_to_ix), embedding, dropout_p=0.1, max_length=MAX_LENGTH).to(device)

# 假设 MAX_LENGTH 是一个已定义的常数
MAX_LENGTH = 10

## Encoder

In [65]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Decoder

In [66]:
class AttnDecoderRNN(nn.Module):
    ATTN_TYPE_DOT_PRODUCT = "Dot Product"

    def __init__(self, hidden_size, output_size, embedding, dropout_p=0.1, max_length=10):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding = embedding
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size * 2, output_size)
        self.max_length = max_length

    def cal_attention(self, hidden, encoder_outputs):
        attn_weights = F.softmax(torch.bmm(hidden.transpose(0, 1), encoder_outputs.transpose(0, 1).transpose(1, 2)), dim=-1)
        attn_output = torch.bmm(attn_weights, encoder_outputs.transpose(0, 1))
        concat_output = torch.cat((attn_output.squeeze(1), hidden.squeeze(0)), dim=1)
        return concat_output

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        output, hidden = self.gru(embedded, hidden)
        concat_output = self.cal_attention(hidden, encoder_outputs)
        output = F.log_softmax(self.out(concat_output), dim=1)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


## Train Function

In [67]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[word_to_ix["<BOS>"]]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
        loss += criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di]  # Teacher forcing

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


## Train Iterations Function

In [68]:
import time
import math

# Helper functions for training
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [69]:
import random
def trainIters(encoder, decoder, n_iters, print_every=500):
    start = time.time()
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        input_tensor = torch.tensor([word_to_ix["hello"]], device=device).unsqueeze(0)
        target_tensor = torch.tensor([word_to_ix["world"]], device=device).unsqueeze(0)

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, MAX_LENGTH)

        if iter % print_every == 0:
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters), iter, iter / n_iters * 100, loss))

trainIters(encoder1, attn_decoder1, 10000, print_every=500)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

## Training Process

In [ ]:
hidden_size = 50
word_to_ix = {"<BOS>": 0, "<EOS>": 1, "hello": 2, "world": 3}  # Example vocabulary
embedding = nn.Embedding(len(word_to_ix), hidden_size).to(device)

encoder1 = EncoderRNN(len(word_to_ix), hidden_size, embedding).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, len(word_to_ix), embedding, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 10000, print_every=500)


## Evaluation

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[word_to_ix["<BOS>"]]], device=device)  # SOS token
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == word_to_ix["<EOS>"]:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(word_list[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words


Methods

 Experiments

Results

Conclusion